## Cleaning and Import of the CF Statement, the Income Statement and the Balance Sheet 

This part of the code aim to clean the data. For instance, in the excel file, you may note that there is some blank line. This part of the code has a goal to delete those empty lines. 

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter

all_statements_path = 'mitra.xlsx'
def load_and_clean_statement_df(statements_path, sheet_name):
    df = pd.read_excel(statements_path, sheet_name=sheet_name,index_col=0)
    df = df.replace('-', np.nan)
    df = df.dropna(how='all')
    df = df.fillna(0) 
    return df
inc_df = load_and_clean_statement_df(all_statements_path, 'Income Statement')
ca_df = load_and_clean_statement_df(all_statements_path, 'Cash Flow')
bs_df = load_and_clean_statement_df(all_statements_path, 'Balance Sheet')

In [2]:
inc_df.head()

,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
For the Fiscal Period Ending,,,,,,
Revenue,32.042,57.876,96.52,9.03,22.668,66.997
Total Revenue,32.042,57.876,96.52,9.03,22.668,66.997
Cost Of Goods Sold,2.595,1.571,2.487,3.457,15.724,19.623
Gross Profit,29.447,56.305,94.033,5.573,6.944,47.374
Selling General & Admin Exp.,10.431,10.956,16.313,17.367,14.386,16.775


In [3]:
ca_df.head()

,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
For the Fiscal Period Ending,,,,,,
Net Income,-35,-12.4,-26.6,-92.1,-116.9,-59.6
Depreciation & Amort.,1.536,3.62,5.302,5.697,7.776,11.94
Amort. of Goodwill and Intangibles,0.506,-0.9,0.131,3.546,1.574,0
"Depreciation & Amort., Total",2.042,2.707,5.433,9.243,9.35,11.94
Other Amortization,0.114,0.144,0.344,0.524,1.076,0


In [4]:
bs_df.head()

,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
For the Fiscal Period Ending,,,,,,
Cash And Equivalents,36.19,118.949,49.72,138.675,32.872,28.285
Short Term Investments,0,0,0.046,0.014,0,0
Total Cash & ST Investments,36.19,118.949,49.766,138.689,32.872,28.285
Accounts Receivable,25.429,5.194,22.433,56.758,17.163,67.265
Other Receivables,6.27,2.72,2.049,2.389,1.681,0


## Visualisation of the past FCFF : basic and vertical analysis

This part will be divided into 3 sections: 

    - Computation and visualisation of the Change in NWC. This step is important to compute the past FCFF of the firm
    - Basic analysis of the FCFF: this step gives us a clear visualisation of the past FCFF, computed thanks to the change in NWC among others
    - Vertical analysis of the FCFF: this step allow us to see clearly the importance of the different element that compose the FCFF

### 1. Change in NWC: computation and visualisation


This first part of the code compute the change in net working capital of the company. The more the NWC is big, the more the company needs cash to finance the liquidity requirement. In other words, the more the NWC is big, the more the current operating liabities are liquid compared to the current operating assets

In [5]:
Currents_Assets=""
Currents_Liabilities=""
ocl_list=bs_df.loc['Other Current Liabilities']
if 'Unearned Revenue, Current' in bs_df.index:
    ocl_list += bs_df.loc['Unearned Revenue, Current'] 
Net_current_assets=bs_df.loc['  Total Receivables']+bs_df.loc['Inventory']+bs_df.loc['Prepaid Exp.']+bs_df.loc['Other Current Assets']
Net_Current_Liabilities=bs_df.loc['Accounts Payable']+bs_df.loc['Accrued Exp.']+bs_df.loc['Curr. Income Taxes Payable']+ocl_list
NWC=Net_current_assets - Net_Current_Liabilities
Change_nwc = NWC - NWC.shift(1)
Change_nwc = Change_nwc.fillna(0)
Change_nwc

2017-12-31     0.000
2018-12-31     6.519
2019-12-31     6.086
2020-12-31    57.871
2021-12-31   -31.216
2022-12-31    15.636
dtype: float64

This second part of the code is there to compute some ratios related to this NWC. Those ratio are important to compute the future NWC of the company. 

In [6]:
revenue_list=inc_df.loc['Revenue']
ar_list=bs_df.loc['  Total Receivables']
inv_list=bs_df.loc['Inventory']
peoca_list=bs_df.loc['Prepaid Exp.']

ap_list=bs_df.loc['Accounts Payable']
if 'Unearned Revenue, Current' in bs_df.index:
    ocl_list += bs_df.loc['Unearned Revenue, Current']
cost_list=inc_df.loc['Cost Of Goods Sold']
ae_list=bs_df.loc['Accrued Exp.']
oca_list=bs_df.loc['Other Current Assets']
taxe_list=bs_df.loc['Curr. Income Taxes Payable']
dso_list = []
for i in range(len(revenue_list)):
    if revenue_list[i] == 0:
        dso_list.append(0)
    else:
        dso_list.append(365*(ar_list[i]/revenue_list[i]))
        
dih_list = []
for i in range(len(cost_list)):
    if cost_list[i] == 0:
        dih_list.append(0)
    else:
        dih_list.append(365*(inv_list[i]/cost_list[i]))

Other_Current_Assets_list = []
for i in range(len(revenue_list)):
    if revenue_list[i] == 0:
        Other_Current_Assets_list.append(0)
    else:
        Other_Current_Assets_list.append((oca_list[i]/revenue_list[i]))

dpo_list = []
for i in range(len(cost_list)):
    if cost_list[i] == 0:
        dpo_list.append(0)
    else:
        dpo_list.append(365*(ap_list[i]/cost_list[i]))

Accrued_Liabilites_list = []
for i in range(len(revenue_list)):
    if revenue_list[i] == 0:
        Accrued_Liabilites_list.append(0)
    else:
        Accrued_Liabilites_list.append((ae_list[i]/revenue_list[i]))

Other_Current_Liabilities_list = []
for i in range(len(revenue_list)):
    if revenue_list[i] == 0:
        Other_Current_Liabilities_list.append(0)
    else:
        Other_Current_Liabilities_list.append((ocl_list[i]/revenue_list[i]))

Taxes_Payable_list = []
for i in range(len(revenue_list)):
    if revenue_list[i] == 0:
        Taxes_Payable_list.append(0)
    else:
        Taxes_Payable_list.append((taxe_list[i]/revenue_list[i]))

inv_turnover_list = []
for i in range(len(revenue_list)):
    if inv_list[i] == 0:
        inv_turnover_list.append(0)
    else:
        inv_turnover_list.append(inv_list[i]/cost_list[i])

receivable_turnover_list = []
for i in range(len(revenue_list)):
    if ar_list[i] == 0:
        receivable_turnover_list.append(0)
    else:
        receivable_turnover_list.append(ar_list[i]/revenue_list[i])

        
payable_turnover_list = []
for i in range(len(revenue_list)):
    if ap_list[i] == 0:
        payable_turnover_list.append(0)
    else:
        payable_turnover_list.append(ap_list[i]/cost_list[i])
        
prepaid_turnover_list=[]
for i in range(len(revenue_list)):
    if ap_list[i] == 0:
        prepaid_turnover_list.append(0)
    else:
        prepaid_turnover_list.append(peoca_list[i]/revenue_list[i])

This third part provide a clear visualisation of the Change in NWC and of the additional ratios

In [7]:


NWC_df = pd.DataFrame({'Revenue': inc_df.loc['Revenue'],
        'COGS': inc_df.loc['Cost Of Goods Sold'],
        'Currents_Assets':Currents_Assets,
        'Accounts_Receivable': bs_df.loc['  Total Receivables'],
        'Inventory': bs_df.loc['Inventory'],
        'Prepaid_Exp.': bs_df.loc['Prepaid Exp.'],
        'Other_Current_Assets': bs_df.loc['Other Current Assets'],
        'Net_current_assets':Net_current_assets,
        'Currents_Liabilities':Currents_Liabilities,
        'Accounts Payable':bs_df.loc['Accounts Payable'],
        'AccruedExp.':bs_df.loc['Accrued Exp.'],
        'Curr.IncomeTaxesPayable':bs_df.loc['Curr. Income Taxes Payable'],
        'OtherCurrentLiabilities':ocl_list,
        'Net_Current_Liabilities':Net_Current_Liabilities,
        'NWC':NWC,       
        'Change_nwc':Change_nwc,
        '':'',
    'Account receivables in % of Sales': receivable_turnover_list,
    'DSO': dso_list,
    'Inventory turnover': inv_turnover_list,
    'DIH': dih_list,
    'Prepaid Expenses in % of Sales':prepaid_turnover_list,
    'Other Current Assets': Other_Current_Assets_list,
    'Account payable in % of COGS': payable_turnover_list,              
    'DPO': dpo_list,
    'Accrued_Liabilites in % of Sales':Accrued_Liabilites_list,
    'Other Current Liabilities in % of Sales':Other_Current_Liabilities_list,
    'Taxes Payable in % of Sales':Taxes_Payable_list
       })

NWC_df = NWC_df.T.reset_index(drop=True)
NWC_df.columns = inc_df.columns
NWC_df.index=['Revenue','COGS',
                'Currents Assets','Accounts Receivable',
                'Inventory','Prepaid Exp.','Other Current Assets','Net current assets',
                'Currents Liabilities','Accounts Payable','Accrued Exp.','Curr.Income Taxes Payable','Other Current Liabilities','Net Current Liabilities',
                'NWC','Change_nwc','','Account receivables in % of Sales','DSO','Inventory turnover','DIH','Prepaid Expenses in % of Sales','Other Current Assets in % of Sales','Account payable in % of COGS','DPO','Accrued_Liabilites in % of Sales',
                'Other Current Liabilities in % of Sales','Taxes Payable in % of Sales']
NWC_df


,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
Revenue,32.042,57.876,96.52,9.03,22.668,66.997
COGS,2.595,1.571,2.487,3.457,15.724,19.623
Currents Assets,,,,,,
Accounts Receivable,31.699,7.914,24.482,59.147,18.844,67.265
Inventory,4.141,10.945,16.277,35.382,43.852,50.312
Prepaid Exp.,0.558,0.001,0,1.568,2.312,0
Other Current Assets,1.624,5.553,0.997,3.689,1.509,0.001
Net current assets,38.022,24.413,41.756,99.786,66.517,117.578
Currents Liabilities,,,,,,
Accounts Payable,16.141,13.071,19.449,23.325,16.915,58.082


### 2. Computation of the FCFF : visualisation and vertical analysis
The first part of the code aims to compute the FCFF algebrically. 

In [8]:
Gross_profit=(inc_df.loc['Revenue']-inc_df.loc['Cost Of Goods Sold'])
Opex = inc_df.loc['Selling General & Admin Exp.'] + inc_df.loc['R & D Exp.'] + inc_df.loc['Other Operating Expense/(Income)'] 
Ebit_= Gross_profit - Opex
NOPAT= Ebit_-(inc_df.loc['Income Tax Expense'])
CFO=NOPAT - Change_nwc + ca_df.loc['Depreciation & Amort., Total']


if 'Stock-Based Compensation' in ca_df.index:
    CFO += ca_df.loc['Stock-Based Compensation']

if '(Gain) Loss On Sale Of Invest.' in ca_df.index:
    CFO += ca_df.loc['(Gain) Loss On Sale Of Invest.']
    
if 'Asset Writedown & Restructuring Costs' in ca_df.index:
    CFO += ca_df.loc['Asset Writedown & Restructuring Costs']
    
if 'Other Operating Activities' in ca_df.index:
    CFO += ca_df.loc['Other Operating Activities']
FCFF=CFO- ca_df.loc['Capital Expenditure']

This second part of the code aims to include the series into a dataframe to visualise clearly the FCFF computation. 

In [9]:

FCFF_bis_df = pd.DataFrame({'Revenue': inc_df.loc['Revenue'],
        'COGS': inc_df.loc['Cost Of Goods Sold'],
        'Gross_profit':Gross_profit,
        'SGA': inc_df.loc['Selling General & Admin Exp.'],
        'RD': inc_df.loc['R & D Exp.'],
        'Other_Operating_Expense': inc_df.loc['Other Operating Expense/(Income)'],
        'DA': ca_df.loc['Depreciation & Amort., Total'],
        'Opex':Opex,
        'Ebit_':Ebit_,
        'Income_Tax_Expense': inc_df.loc['Income Tax Expense'],
        'Net_income':NOPAT,
        'Depreciation_and_Amortization': ca_df.loc['Depreciation & Amort., Total'],
        'Stock-BasedCompensation': ca_df.loc['Stock-Based Compensation'],
        'OtherOperatingActivities': ca_df.loc['Other Operating Activities'],
        '(Gain)LossOnSaleOfInvest.': ca_df.get('(Gain) Loss On Sale Of Invest.', 0),
        'Asset_Writedown_Restructuring_Costs': ca_df.get('Asset Writedown & Restructuring Costs', 0),
        'Change_nwc':Change_nwc,
        'CFO': CFO,
        'Capital_Expenditure':ca_df.loc['Capital Expenditure'],
        'FCFF':FCFF
       })
FCFF_bis_df=FCFF_bis_df.T
FCFF_df = FCFF_bis_df.reset_index(drop=True)
FCFF_df.columns = inc_df.columns
FCFF_df.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A','Opex',
                'EBIT','Income tax expenses','Net Income','D&A',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']

FCFF_df



,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
Revenue,32.042,57.876,96.52,9.03,22.668,66.997
COGS,2.595,1.571,2.487,3.457,15.724,19.623
Gross_profit,29.447,56.305,94.033,5.573,6.944,47.374
SG&A,10.431,10.956,16.313,17.367,14.386,16.775
R&D,48.185,35.713,57.073,78.458,85.243,64.041
Other Operating Expense,-3,-4.6,-6.3,-6.6,-4.8,-7.2
D&A,2.042,2.707,5.433,9.243,9.35,11.94
Opex,55.616,42.069,67.086,89.225,94.829,73.616
EBIT,-26.169,14.236,26.947,-83.652,-87.885,-26.242
Income tax expenses,-13.1,-9.9,-4.9,-18.8,-6.9,48.139


### 3. FCFF Vertical Analysis: Computation and Visualisation

This first part aims to compute the vertical analysis. This is done by simply dividing the different label by the revenue of the corresponding year

In [10]:
list_variables = ['Revenue', 'COGS', 'Gross_profit', 'SGA', 'RD',
       'Other_Operating_Expense', 'DA', 'Opex', 'Ebit_', 'Income_Tax_Expense',
       'Net_income', 'Depreciation_and_Amortization',
       'Stock-BasedCompensation', 'OtherOperatingActivities',
       '(Gain)LossOnSaleOfInvest.', 'Asset_Writedown_Restructuring_Costs',
       'Change_nwc', 'CFO', 'Capital_Expenditure', 'FCFF']
variables = []

for variable in list_variables:
    x = FCFF_bis_df.loc[variable]/FCFF_bis_df.loc['Revenue']
    variables.append(x)
    


This second part put the series into a dataframe to afford a clear visualisation

In [11]:
FCFF_bis_vertical=pd.DataFrame(variables)
FCFF_bis_vertical.index=['Revenue', 'COGS', 'Gross_profit', 'SGA', 'RD',
       'Other_Operating_Expense', 'DA', 'Opex', 'Ebit_', 'Income_Tax_Expense',
       'Net_income', 'Depreciation_and_Amortization',
       'Stock-BasedCompensation', 'OtherOperatingActivities',
       '(Gain)LossOnSaleOfInvest.', 'Asset_Writedown_Restructuring_Costs',
       'Change_nwc', 'CFO', 'Capital_Expenditure', 'FCFF']
FCFF_vertical=FCFF_bis_vertical.reset_index(drop=True)
FCFF_vertical.columns = inc_df.columns
FCFF_vertical.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A','Opex',
                'EBIT','Income tax expenses','Net Income','D&A',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']

FCFF_vertical

,2017-12-31,2018-12-31,2019-12-31,2020-12-31,2021-12-31,2022-12-31
Revenue,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
COGS,0.080987,0.027144,0.025767,0.382835,0.693665,0.292894
Gross_profit,0.919013,0.972856,0.974233,0.617165,0.306335,0.707106
SG&A,0.325541,0.189301,0.169012,1.923256,0.634639,0.250384
R&D,1.503808,0.617061,0.591308,8.688594,3.760499,0.955879
Other Operating Expense,-0.093627,-0.079480,-0.065271,-0.730897,-0.211752,-0.107467
D&A,0.063729,0.046772,0.056289,1.023588,0.412476,0.178217
Opex,1.735722,0.726882,0.695048,9.880952,4.183386,1.098795
EBIT,-0.816709,0.245974,0.279186,-9.263787,-3.877051,-0.391689
Income tax expenses,-0.408838,-0.171055,-0.050767,-2.081949,-0.304394,0.718525


In [12]:
variables

[2017-12-31    1.0
 2018-12-31    1.0
 2019-12-31    1.0
 2020-12-31    1.0
 2021-12-31    1.0
 2022-12-31    1.0
 Name: Revenue, dtype: object,
 2017-12-31    0.080987
 2018-12-31    0.027144
 2019-12-31    0.025767
 2020-12-31    0.382835
 2021-12-31    0.693665
 2022-12-31    0.292894
 dtype: object,
 2017-12-31    0.919013
 2018-12-31    0.972856
 2019-12-31    0.974233
 2020-12-31    0.617165
 2021-12-31    0.306335
 2022-12-31    0.707106
 dtype: object,
 2017-12-31    0.325541
 2018-12-31    0.189301
 2019-12-31    0.169012
 2020-12-31    1.923256
 2021-12-31    0.634639
 2022-12-31    0.250384
 dtype: object,
 2017-12-31    1.503808
 2018-12-31    0.617061
 2019-12-31    0.591308
 2020-12-31    8.688594
 2021-12-31    3.760499
 2022-12-31    0.955879
 dtype: object,
 2017-12-31   -0.093627
 2018-12-31    -0.07948
 2019-12-31   -0.065271
 2020-12-31   -0.730897
 2021-12-31   -0.211752
 2022-12-31   -0.107467
 dtype: object,
 2017-12-31    0.063729
 2018-12-31    0.046772
 2019-1

## Statistics on past FCFF

3 statistics tables will be given in this part: 

    - A statistic table concerning the NWC ratios
    - A statistic table concerning the FCFF basic analysis
    - A statistic table concerning the FCFF vertical analysis

### NWC ratios statistics

In [13]:
import statistics as stat
avg_acr = stat.mean(receivable_turnover_list)
avg_dso = stat.mean(dso_list)
avg_inv = stat.mean(inv_turnover_list)
avg_dih = stat.mean(dih_list)
avg_pre = stat.mean(prepaid_turnover_list)
avg_oca = stat.mean(Other_Current_Assets_list)
avg_acp = stat.mean(payable_turnover_list)
avg_dpo = stat.mean(dpo_list)
avg_acl = stat.mean(Accrued_Liabilites_list)
avg_ocl = stat.mean(Other_Current_Liabilities_list)
avg_taxe = stat.mean(Taxes_Payable_list)

median_acr = stat.mean(receivable_turnover_list)
median_dso = stat.median(dso_list)
median_inv = stat.median(inv_turnover_list)
median_dih = stat.median(dih_list)
median_pre = stat.median(prepaid_turnover_list)
median_oca = stat.median(Other_Current_Assets_list)
median_acp = stat.mean(payable_turnover_list)
median_dpo = stat.median(dpo_list)
median_acl = stat.median(Accrued_Liabilites_list)
median_ocl = stat.median(Other_Current_Liabilities_list)
median_taxe = stat.median(Taxes_Payable_list)

last_acr = receivable_turnover_list[-1]
last_dso = dso_list[-1]
last_inv = inv_turnover_list[-1]
last_dih = dih_list[-1]
last_pre = prepaid_turnover_list[-1]
last_oca = Other_Current_Assets_list[-1]
last_acp = payable_turnover_list[-1]
last_dpo = dpo_list[-1]
last_acl = Accrued_Liabilites_list[-1]
last_ocl = Other_Current_Liabilities_list[-1]
last_taxe = Taxes_Payable_list[-1]
stat_NWC_df = pd.DataFrame({
    'Average': [avg_acr,avg_dso, avg_inv, avg_dih,avg_pre, avg_oca, avg_acp, avg_dpo, avg_acl, avg_ocl, avg_taxe],
    'Median': [median_acr, median_dso, median_inv, median_dih, median_pre, median_oca, median_acp, median_dpo, median_acl, median_ocl, median_taxe],
    'Last Value': [last_acr, last_dso, last_inv, last_dih, last_pre, last_oca, last_acr, last_dpo, last_acl, last_ocl, last_taxe]
})
stat_NWC_df.index=['Account receivables in % of Sales','DSO','Inventory Turnover','DIH', 'Prepaid Expenses in % of Sales', 'Other Current Assets in % of Sales','Account payable in % of COGS','DPO','Accrued_Liabilites in % of Sales',
                'Other Current Liabilities in % of Sales','Taxes Payable in % of Sales']
stat_NWC_df

,Average,Median,Last Value
Account receivables in % of Sales,1.627507,1.627507,1.004000
DSO,594.040081,332.259380,366.460065
Inventory Turnover,5.115861,4.666845,2.563930
DIH,1867.289380,1703.398595,935.834480
Prepaid Expenses in % of Sales,0.048845,0.008716,0.000000
Other Current Assets in % of Sales,0.105345,0.058627,0.000015
Account payable in % of COGS,5.523883,5.523883,1.004000
DPO,2016.217409,2366.517316,1080.361311
Accrued_Liabilites in % of Sales,0.061203,0.024090,0.000000
Other Current Liabilities in % of Sales,0.264016,0.139538,0.038241


In [14]:
list_variables = ['Revenue', 'COGS', 'Gross_profit', 'SGA', 'RD',
       'Other_Operating_Expense', 'DA', 'Opex', 'Ebit_', 'Income_Tax_Expense',
       'Net_income', 'Depreciation_and_Amortization',
       'Stock-BasedCompensation', 'OtherOperatingActivities',
       '(Gain)LossOnSaleOfInvest.', 'Asset_Writedown_Restructuring_Costs',
       'Change_nwc', 'CFO', 'Capital_Expenditure', 'FCFF']
avg_variable=[]
median_variable=[]
last_variable=[]
for variables in list_variables: 
    variables_list=FCFF_bis_df.loc[variables]
    avg_variable.append(stat.mean(variables_list))
    median_variable.append(stat.median(variables_list))
    last_variable.append(variables_list[-1])
stat_FCFF_df = pd.DataFrame({
    'Average': avg_variable,
    'Median': median_variable,
    'Last Value': last_variable
}) 
stat_FCFF_df.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A','Opex',
                'EBIT','Income tax expenses','Net Income','D&A',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']
stat_FCFF_df

,Average,Median,Last Value
Revenue,47.522167,44.9590,66.997
COGS,7.576167,3.0260,19.623
Gross_profit,39.946000,38.4105,47.374
SG&A,14.371333,15.3495,16.775
R&D,61.452167,60.5570,64.041
Other Operating Expense,-5.416667,-5.5500,-7.200
D&A,6.785833,7.3380,11.940
Opex,70.406833,70.3510,73.616
EBIT,-30.460833,-26.2055,-26.242
Income tax expenses,-0.910167,-8.4000,48.139


In [15]:
list_variables = ['Revenue', 'COGS', 'Gross_profit', 'SGA', 'RD',
       'Other_Operating_Expense', 'DA', 'Opex', 'Ebit_', 'Income_Tax_Expense',
       'Net_income', 'Depreciation_and_Amortization',
       'Stock-BasedCompensation', 'OtherOperatingActivities',
       '(Gain)LossOnSaleOfInvest.', 'Asset_Writedown_Restructuring_Costs',
       'Change_nwc', 'CFO', 'Capital_Expenditure', 'FCFF']
avg_pct_variable=[]
median_pct_variable=[]
last_pct_variable=[]
for variables in list_variables: 
    variables_list=FCFF_bis_vertical.loc[variables]
    avg_pct_variable.append(stat.mean(variables_list))
    median_pct_variable.append(stat.median(variables_list))
    last_pct_variable.append(variables_list[-1])
stat_FCFF_vertical_df = pd.DataFrame({
    'Average': avg_pct_variable,
    'Median': median_pct_variable,
    'Last Value': last_pct_variable
}) 
stat_FCFF_vertical_df.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A','Opex',
                'EBIT','Income tax expenses','Net Income','D&A',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']

In [16]:
Assumption_revenues_list=[]
Assumption_revenues_df=pd.DataFrame(Assumption_revenues_list)
Assumption_revenues_df.index=['2023','2024','2025','2026','2027','2028']
labels_revenues=['Sales Estelle','License Estelle','Sales Donesta','License Donesta','Sales Myring','License Myring','Others']
Assumption_revenues_df['Estelle']=[47.93,64.14,113.51,92.43,132.64,192.96]
Assumption_revenues_df['Donesta']=[15,12.8,34.9,14.2,31.1,33.3]
Assumption_revenues_df['Myring']=[7.14,8.94,14.92,10.64,11.79,12.74]
Assumption_revenues_df['Other']=[0.92,1.41,2.15,3.29,5.04,7.72]
total_sales_list=[]
for year in Assumption_revenues_df.index:
    total_sales_list.append(sum(Assumption_revenues_df.T[year]))
    
Assumption_revenues_df['Total']=total_sales_list
Assumption_revenues_df=Assumption_revenues_df.T

In [17]:
stat.mean(FCFF_vertical.loc['SG&A',['2017-12-31','2018-12-31','2019-12-31','2021-12-31','2022-12-31']])

0.31377556596784495

In [18]:
choice_list=[]
draft_futurV_FCFF_df=pd.DataFrame(choice_list)

draft_futurV_FCFF_df.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A1','Opex',
                'EBIT','Income tax expenses','Net Income','D&A2',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']
year_to_forecast=['2023','2024','2025','2026','2027','2028']

for year in year_to_forecast:
    draft_futurV_FCFF_df[year]=avg_pct_variable
    draft_futurV_FCFF_df[year]['SG&A']=stat.mean(FCFF_vertical.loc['SG&A',['2017-12-31','2018-12-31','2019-12-31','2021-12-31','2022-12-31']])
    draft_futurV_FCFF_df[year]['R&D']=stat.mean(FCFF_vertical.loc['R&D',['2018-12-31','2019-12-31']])
    draft_futurV_FCFF_df[year]['Other Operating Expense']=stat.mean(FCFF_vertical.loc['Other Operating Expense',['2017-12-31','2018-12-31','2019-12-31','2021-12-31','2022-12-31']])
    draft_futurV_FCFF_df[year]['Opex']=sum([draft_futurV_FCFF_df[year]['SG&A'],draft_futurV_FCFF_df[year]['R&D'],draft_futurV_FCFF_df[year]['Other Operating Expense']])
    draft_futurV_FCFF_df[year]['Capital Expenditure']=stat.mean(FCFF_vertical.loc['Capital Expenditure',['2017-12-31','2018-12-31','2019-12-31','2021-12-31','2022-12-31']])
    draft_futurV_FCFF_df[year]['D&A1']=stat.mean(FCFF_bis_vertical.loc['DA',['2019-12-31','2020-12-31','2021-12-31',]]/-FCFF_vertical.loc['Capital Expenditure',['2019-12-31','2020-12-31','2021-12-31']])
    draft_futurV_FCFF_df[year]['D&A2']=stat.mean(FCFF_bis_vertical.loc['DA',['2019-12-31','2020-12-31','2021-12-31',]]/-FCFF_vertical.loc['Capital Expenditure',['2019-12-31','2020-12-31','2021-12-31']])

draft_futurV_FCFF_df.loc['COGS']=[0.2874, 0.2662, 0.1834, 0.2083, 0.1627, 0.1289]
draft_futurV_FCFF_df.loc['Income tax expenses']=stat.median(np.absolute(FCFF_df.loc['Income tax expenses',['2017-12-31','2018-12-31','2019-12-31','2020-12-31','2021-12-31']]/FCFF_df.loc['EBIT',['2017-12-31','2018-12-31','2019-12-31','2020-12-31','2021-12-31']]))



draft_futurV_FCFF_df

,2023,2024,2025,2026,2027,2028
Revenue,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
COGS,0.287400,0.266200,0.183400,0.208300,0.162700,0.128900
Gross_profit,0.749451,0.749451,0.749451,0.749451,0.749451,0.749451
SG&A,0.313776,0.313776,0.313776,0.313776,0.313776,0.313776
R&D,0.604184,0.604184,0.604184,0.604184,0.604184,0.604184
Other Operating Expense,-0.111520,-0.111520,-0.111520,-0.111520,-0.111520,-0.111520
D&A1,0.724828,0.724828,0.724828,0.724828,0.724828,0.724828
Opex,0.806440,0.806440,0.806440,0.806440,0.806440,0.806440
EBIT,-2.304013,-2.304013,-2.304013,-2.304013,-2.304013,-2.304013
Income tax expenses,0.224741,0.224741,0.224741,0.224741,0.224741,0.224741


In [19]:

forecasted_revenue_list=Assumption_revenues_df.loc['Total']
forecasted_cost_list=draft_futurV_FCFF_df.loc['COGS']*Assumption_revenues_df.loc['Total']
forecasted_ar_list=stat_NWC_df.loc['Account receivables in % of Sales','Average']*forecasted_revenue_list
forecasted_inv_list=stat_NWC_df.loc['Inventory Turnover','Average']*forecasted_cost_list
forecasted_peoca_list=stat_NWC_df.loc['Prepaid Expenses in % of Sales','Average']*forecasted_revenue_list
forecasted_ap_list=stat_NWC_df.loc['Account payable in % of COGS','Average']*forecasted_cost_list
forecasted_ae_list=stat_NWC_df.loc['Accrued_Liabilites in % of Sales','Average']*forecasted_revenue_list
forecasted_oca_list=stat_NWC_df.loc['Other Current Assets in % of Sales','Average']*forecasted_revenue_list
forecasted_taxe_list=stat_NWC_df.loc['Taxes Payable in % of Sales','Average']*forecasted_revenue_list
forecasted_ocl_list=stat_NWC_df.loc['Other Current Liabilities in % of Sales','Average']*forecasted_revenue_list
forecasted_net_current_assets=forecasted_ar_list+forecasted_inv_list+forecasted_peoca_list+forecasted_oca_list
forecasted_net_current_liabilities=forecasted_ap_list+forecasted_ae_list+forecasted_taxe_list+forecasted_ocl_list
forecasted_NWC=forecasted_net_current_assets-forecasted_net_current_liabilities
forecasted_change_NWC= forecasted_NWC - forecasted_NWC.shift(1)
forecasted_change_NWC_2023=forecasted_NWC['2023']-NWC['2022-12-31']
forecasted_change_NWC=forecasted_change_NWC.fillna(forecasted_change_NWC_2023)
forecasted_NWC_df = pd.DataFrame({'Revenue': forecasted_revenue_list,
        'COGS': forecasted_cost_list,
        'Currents_Assets':Currents_Assets,
        'Accounts_Receivable': forecasted_ar_list,
        'Inventory': forecasted_inv_list,
        'Prepaid_Exp.': forecasted_peoca_list,
        'Other_Current_Assets': forecasted_oca_list,
        'Net_current_assets':forecasted_net_current_assets,
        'Currents_Liabilities':Currents_Liabilities,
        'Accounts Payable':forecasted_ap_list,
        'AccruedExp.':forecasted_ae_list,
        'Curr.IncomeTaxesPayable':forecasted_taxe_list,
        'OtherCurrentLiabilities':forecasted_ocl_list,
        'Net_Current_Liabilities':forecasted_net_current_liabilities,
        'NWC':forecasted_NWC,       
        'Change_nwc':forecasted_change_NWC})
forecasted_NWC_df=forecasted_NWC_df.T
forecasted_NWC_df

,2023,2024,2025,2026,2027,2028
Revenue,70.99,87.29,165.48,120.56,180.57,246.72
COGS,20.402526,23.236598,30.349032,25.112648,29.378739,31.802208
Currents_Assets,,,,,,
Accounts_Receivable,115.536727,142.065092,269.31987,196.212252,293.878952,401.538544
Inventory,104.376494,118.875213,155.261439,128.472824,150.297554,162.695686
Prepaid_Exp.,3.467499,4.26367,8.082852,5.88874,8.819922,12.051011
Other_Current_Assets,7.478455,9.195581,17.432522,12.700416,19.02218,25.990764
Net_current_assets,230.859174,274.399557,450.096682,343.274232,472.018608,602.276005
Currents_Liabilities,,,,,,
Accounts Payable,112.701173,128.356256,167.644511,138.719337,162.284726,175.671686


In [20]:
list_variables = ['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A1','Opex',
                'EBIT','Income tax expenses','Net Income','D&A2',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']
variables = []

for variable in list_variables:
    x = draft_futurV_FCFF_df.loc[variable]*Assumption_revenues_df.loc['Total']
    variables.append(x)
Futur_FCFF_df=pd.DataFrame(variables)
Futur_FCFF_df.columns = draft_futurV_FCFF_df.columns

Futur_FCFF_df.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A','Opex',
                'EBIT','Income tax expenses','NOPAT','D&A',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']
Futur_FCFF_df.loc['Change nwc']=forecasted_NWC_df.loc['Change_nwc']
Futur_FCFF_df.loc['Gross_profit']=Futur_FCFF_df.loc['Revenue']-Futur_FCFF_df.loc['COGS']
Futur_FCFF_df.loc['EBIT']=Futur_FCFF_df.loc['Gross_profit']-Futur_FCFF_df.loc['Opex']
Futur_FCFF_df.loc['NOPAT']=Futur_FCFF_df.loc['EBIT']-Futur_FCFF_df.loc['Income tax expenses']
Futur_FCFF_df.loc['CFO']=Futur_FCFF_df.loc['NOPAT']+Futur_FCFF_df.loc['Stock Based Compensation']+Futur_FCFF_df.loc['(Gain) Loss On Sale Of Invest']+Futur_FCFF_df.loc['Asset Writedown Restructuring Costs']-Futur_FCFF_df.loc['Change nwc']
Futur_FCFF_df.loc['FCFF']=Futur_FCFF_df.loc['CFO']-Futur_FCFF_df.loc['Capital Expenditure']
Futur_FCFF_df.loc['Income tax expenses']=draft_futurV_FCFF_df.loc['Income tax expenses']*Futur_FCFF_df.loc['EBIT']

Futur_FCFF_df


,2023,2024,2025,2026,2027,2028
Revenue,70.990000,87.290000,165.480000,120.560000,180.570000,246.720000
COGS,20.402526,23.236598,30.349032,25.112648,29.378739,31.802208
Gross_profit,50.587474,64.053402,135.130968,95.447352,151.191261,214.917792
SG&A,22.274927,27.389469,51.923581,37.828782,56.658454,77.414708
R&D,42.891026,52.739226,99.980378,72.840430,109.097515,149.064290
Other Operating Expense,-7.916785,-9.734556,-18.454283,-13.444817,-20.137115,-27.514144
D&A,51.455541,63.270238,119.944541,87.385266,130.882196,178.829570
Opex,57.249169,70.394139,133.449676,97.224395,145.618854,198.964854
EBIT,-6.661695,-6.340737,1.681292,-1.777043,5.572407,15.952938
Income tax expenses,-1.497153,-1.425021,0.377855,-0.399374,1.252346,3.585273


In [21]:
Futur_FCFF_df.columns = draft_futurV_FCFF_df.columns

Futur_FCFF_df.index=['Revenue','COGS','Gross_profit','SG&A','R&D','Other Operating Expense','D&A','Opex',
                'EBIT','Income tax expenses','Net Income','D&A',
                'Stock Based Compensation','Other Operating Activities',
                '(Gain) Loss On Sale Of Invest',
                'Asset Writedown Restructuring Costs',
                'Change nwc','CFO','Capital Expenditure','FCFF']
Futur_FCFF_df

,2023,2024,2025,2026,2027,2028
Revenue,70.990000,87.290000,165.480000,120.560000,180.570000,246.720000
COGS,20.402526,23.236598,30.349032,25.112648,29.378739,31.802208
Gross_profit,50.587474,64.053402,135.130968,95.447352,151.191261,214.917792
SG&A,22.274927,27.389469,51.923581,37.828782,56.658454,77.414708
R&D,42.891026,52.739226,99.980378,72.840430,109.097515,149.064290
Other Operating Expense,-7.916785,-9.734556,-18.454283,-13.444817,-20.137115,-27.514144
D&A,51.455541,63.270238,119.944541,87.385266,130.882196,178.829570
Opex,57.249169,70.394139,133.449676,97.224395,145.618854,198.964854
EBIT,-6.661695,-6.340737,1.681292,-1.777043,5.572407,15.952938
Income tax expenses,-1.497153,-1.425021,0.377855,-0.399374,1.252346,3.585273


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# Send a GET request to the website URL
url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ratings.html"
response = requests.get(url,verify=False)

soup = BeautifulSoup(response.content, 'html.parser')

table = soup.find('table')

rows = table.find_all('tr')

data = []
for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)
df = pd.DataFrame(data)
df.columns=(df.iloc[0])
df=df.drop(index=[0,16])
df.iloc[:, 0:1] = df.iloc[:, 0:1].astype(float)
df.iloc[:, 3] = (df.iloc[:, 3].astype(str).str.replace('%', '').astype(float))/100
df

/Users/gavysingh/opt/anaconda3/envs/DataManagement/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pages.stern.nyu.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/var/folders/db/5lqty4n965qf1vnzdn73_tzw0000gn/T/ipykernel_392/2250110029.py:23: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 3] = (df.iloc[:, 3].astype(str).str.replace('%', '').astype(float))/100


,>,≤ to,Rating is,Spread is
1,-100000.0,0.199999,D2/D,0.2000
2,0.2,0.649999,C2/C,0.1750
3,0.65,0.799999,Ca2/CC,0.1578
4,0.8,1.249999,Caa/CCC,0.1157
5,1.25,1.499999,B3/B-,0.0737
6,1.5,1.749999,B2/B,0.0526
7,1.75,1.999999,B1/B+,0.0455
8,2.0,2.2499999,Ba2/BB,0.0313
9,2.25,2.49999,Ba1/BB+,0.0242
10,2.5,2.999999,Baa2/BBB,0.0200


In [23]:
interest=inc_df.loc['  Net Interest Exp.']
ratio=Ebit_[-1]/interest[-1]
ratio

1.929558823529412

In [24]:
i=0
while i < len(df.index):
    if float(df.iloc[i, 0]) < ratio < float(df.iloc[i, 1]):
        spread = df.iloc[i, 3]
    i+=1
spread

0.0455

In [25]:
import yfinance as yf
url = "http://www.worldgovernmentbonds.com/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
country='Belgium'
table = soup.find("table", {"class": "homeBondTable"})
for row in table.find_all("tr"):
    if country in row.text:
        rf = row.find("td", {"class": "w3-right-align w3-bold"}).text.strip()
        rf = (float(rf.strip("%")))/100
        break  
tot_debt=510330000
year=2023
debts = [
    [2023, 25796],
    [2024, 22996],
    [2025, 105761],
    [2028, 301637],
    [2033, 53141],
]
total_debt = sum(debt[1] for debt in debts)
w_avg_maturity = sum((debt[0] - year) * (debt[1] / total_debt) for debt in debts)
int_exp=inc_df.loc['Interest Expense']
int_exp = abs(int_exp[-1])
cost_of_debt=rf + spread
step_1 = (1 - (1/(1+cost_of_debt)**w_avg_maturity))/cost_of_debt
step_2 = tot_debt/(1+cost_of_debt)**w_avg_maturity
mv_debt = int_exp * step_1 + step_2
taxe=draft_futurV_FCFF_df.loc['Income tax expenses'][0]

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# Send a GET request to the website URL
url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html"
response = requests.get(url,verify=False)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find("table", {"width": "919"})
for row in table.find_all("tr"):
    if country in row.text:
        erp = row.find_all("td")[2].text.strip()
        erp = (float(erp.strip("%")))/100
        break
erp

/Users/gavysingh/opt/anaconda3/envs/DataManagement/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pages.stern.nyu.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


0.0697

In [27]:
mitra = yf.Ticker('MITRA.BR')
marketCap = mitra.info['marketCap']

In [28]:
beta=2.13

In [29]:
cost_of_equity = beta * erp + rf
cost_of_debt = rf + spread
cap_structure = marketCap + mv_debt
w_cost_equity = marketCap / cap_structure
w_cost_debt = mv_debt / cap_structure
wacc = cost_of_equity * w_cost_equity + cost_of_debt * w_cost_debt

In [30]:
df = pd.DataFrame({
    'cost of equity':[cost_of_equity],
    'beta': [beta],
    'erp': [erp],
    'Market Value of Equity': [marketCap],
    'Weight of Equity':[w_cost_equity],
    'cost of debt':[cost_of_debt],
    'rf': [rf],
    'Spread': [spread],
    'Taxe rate':[taxe],
    'Market Value of Debt':[mv_debt],
    'Weight of Debt':[w_cost_debt],
    'WACC':[wacc]
})

df.T.style.format("{:.3f}")

,0
cost of equity,0.180
beta,2.130
erp,0.070
Market Value of Equity,163515872.000
Weight of Equity,0.308
cost of debt,0.077
rf,0.031
Spread,0.045
Taxe rate,0.225
Market Value of Debt,366781465.757


In [40]:
import base64